## Dependencies

In [1]:
import warnings
import pandas as pd
import seaborn as sns
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from tensorflow.keras import Model, optimizers
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Concatenate, BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from googleqa_utilityscript import *


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
hold_out = pd.read_csv('/kaggle/input/googleqa-dataset/hold-out.csv')
train = hold_out[hold_out['set'] == 'train']
validation = hold_out[hold_out['set'] == 'validation']
test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Train samples: %s' % len(train))
print('Validation samples: %s' % len(validation))
print('Test samples: %s' % len(test))
display(train.head())

Train samples: 4863
Validation samples: 1216
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written,set
0,6148,Important non-technical course for programmers?,What kind of non-technical training course do ...,Louis Rhys,https://programmers.stackexchange.com/users/8486,Business\n\nThe biggest problem I've seen with...,Ryan Hayes,https://programmers.stackexchange.com/users/1521,http://programmers.stackexchange.com/questions...,TECHNOLOGY,...,1.000000,0.666667,1.0,1.0,0.933333,0.000000,0.000000,1.0,0.888889,train
1,3971,Water : Aquatic :: Sand : xxx?,Just as aquatic is to water and aerial is to a...,coleopterist,https://english.stackexchange.com/users/23608,"Well, fancy words for ‘sandy’ are arenarious a...",tchrist,https://english.stackexchange.com/users/2085,http://english.stackexchange.com/questions/938...,CULTURE,...,1.000000,1.000000,1.0,1.0,1.000000,0.000000,0.000000,1.0,0.888889,train
2,4367,"Learning the musical concepts in the book ""Göd...","I am reading the book ""Gödel, Escher, Bach"", i...",Otavio Macedo,https://music.stackexchange.com/users/12666,I agree with luser droog's answer.\n\nMy under...,luser droog,https://music.stackexchange.com/users/1344,http://music.stackexchange.com/questions/22216...,LIFE_ARTS,...,1.000000,0.666667,1.0,1.0,0.900000,0.333333,0.333333,1.0,0.777778,train
3,6605,HowTo: Add Class to Sidebar Widget List-Items,The newest version of Bootstrap (v3.0) adds a ...,sleeper,https://wordpress.stackexchange.com/users/12116,Update: I fixed the debug warning and everythi...,sleeper,https://wordpress.stackexchange.com/users/12116,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,...,0.777778,0.555556,1.0,1.0,0.733333,0.500000,0.000000,0.5,0.888889,train
4,6522,MVC4 jQuery UI does not work,I cannot get jQuery UI working in my ASP.NET M...,Petr,https://stackoverflow.com/users/958557,"In the layout.cshtml view, move \n\n@Scripts.R...",Yuri Morales,https://stackoverflow.com/users/1459163,http://stackoverflow.com/questions/13112519/mv...,STACKOVERFLOW,...,1.000000,0.666667,1.0,1.0,1.000000,0.000000,1.000000,0.0,1.000000,train


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Train/validation sets

In [4]:
# Train features
X_train_title = train['question_title']
X_train_body = train['question_body']
X_train_answer = train['answer']

X_train = [X_train_title, X_train_body, X_train_answer]
Y_train = train[target_cols].values

# Validation features
X_valid_title = validation['question_title']
X_valid_body = validation['question_body']
X_valid_answer = validation['answer']

X_valid = [X_valid_title, X_valid_body, X_valid_answer]
Y_valid = validation[target_cols].values

print('Train samples: %d' % len(Y_train))
print('Validation samples: %d' % len(Y_valid))

Train samples: 4863
Validation samples: 1216


# Model parameters

In [5]:
EPOCHS = 30
BATCH_SIZE = 16
LEARNING_RATE = 3e-4
EMBEDDDING_SIZE = 512
N_CLASS = len(target_cols)
ES_PATIENCE = 5
RLROP_PATIENCE = 2
DECAY_DROP = 0.5
module_url = '../input/universalsentenceencodermodels/universal-sentence-encoder-models/use-large'
model_path = '../working/use_baseline.h5'

# Model

In [6]:
es = EarlyStopping(monitor='val_loss', mode='min', patience=ES_PATIENCE, restore_best_weights=True, verbose=1)
rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', patience=RLROP_PATIENCE, factor=DECAY_DROP, min_lr=1e-6, verbose=1)
use_embed = hub.load(module_url)

def USEEmbedding(x):
    return use_embed(tf.squeeze(tf.cast(x, tf.string)))

In [7]:
input_title = Input(shape=(1,), dtype=tf.string, name='input_title')
embedding_title = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_title)

input_body = Input(shape=(1,), dtype=tf.string, name='input_body')
embedding_body = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_body)

input_answer = Input(shape=(1,), dtype=tf.string, name='input_answer')
embedding_answer = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_answer)

x = Concatenate()([embedding_title, embedding_body, embedding_answer])
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(N_CLASS, activation='sigmoid', name='output')(x)
model = Model(inputs=[input_title, input_body, input_answer], outputs=[output])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_title (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_body (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_answer (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 512)          0           input_title[0][0]                
______________________________________________________________________________________________

# Train model

In [8]:
optimizer = optimizers.Adam(LEARNING_RATE)
callback_list = [es, rlrop, SpearmanRhoCallback(training_data=(X_train, Y_train), validation_data=(X_valid, Y_valid))]
model.compile(optimizer=optimizer, loss='binary_crossentropy')

history = model.fit(X_train, Y_train, 
                    validation_data=(X_valid, Y_valid), 
                    callbacks=callback_list, 
                    batch_size=BATCH_SIZE, 
                    epochs=EPOCHS, 
                    verbose=2).history

Train on 4863 samples, validate on 1216 samples
Epoch 1/30
Train spearman-rho: 0.2809 Validation spearman-rho: 0.2671
4863/4863 - 292s - loss: 0.4652 - val_loss: 0.3991
Epoch 2/30
Train spearman-rho: 0.3497 Validation spearman-rho: 0.3274
4863/4863 - 138s - loss: 0.3971 - val_loss: 0.3846
Epoch 3/30
Train spearman-rho: 0.3806 Validation spearman-rho: 0.3505
4863/4863 - 139s - loss: 0.3873 - val_loss: 0.3782
Epoch 4/30
Train spearman-rho: 0.3997 Validation spearman-rho: 0.3602
4863/4863 - 138s - loss: 0.3813 - val_loss: 0.3751
Epoch 5/30
Train spearman-rho: 0.4144 Validation spearman-rho: 0.3673
4863/4863 - 139s - loss: 0.3779 - val_loss: 0.3734
Epoch 6/30
Train spearman-rho: 0.4246 Validation spearman-rho: 0.3717
4863/4863 - 138s - loss: 0.3754 - val_loss: 0.3721
Epoch 7/30
Train spearman-rho: 0.4353 Validation spearman-rho: 0.3750
4863/4863 - 138s - loss: 0.3730 - val_loss: 0.3714
Epoch 8/30
Train spearman-rho: 0.4430 Validation spearman-rho: 0.3781
4863/4863 - 138s - loss: 0.3709 - v

#### Save model

In [9]:
model.save_weights(model_path)

# Evaluation

In [10]:
preds_train = model.predict(X_train)
preds_val = model.predict(X_valid)

rho_train = np.mean([spearmanr(Y_train[:, ind], preds_train[:, ind] + np.random.normal(0, 1e-7, preds_train.shape[0])).correlation for ind in range(preds_train.shape[1])])
rho_val = np.mean([spearmanr(Y_valid[:, ind], preds_val[:, ind] + np.random.normal(0, 1e-7, preds_val.shape[0])).correlation for ind in range(preds_val.shape[1])])

print('Train spearman-rho: %.3f' % rho_train)
print('Validation spearman-rho: %.3f' % rho_val)

Train spearman-rho: 0.494
Validation spearman-rho: 0.392


# Make predictions on test

In [11]:
# Test features
X_test_title = test['question_title']
X_test_body = test['question_body']
X_test_answer = test['answer']

X_test = [X_test_title, X_test_body, X_test_answer]
Y_test = model.predict(X_test)

In [12]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.896686,0.688717,0.209283,0.759942,0.650028,0.752130,0.585252,0.515229,0.219695,...,0.886336,0.898764,0.636083,0.947691,0.961507,0.841376,0.063132,0.080512,0.865487,0.916164
1,46,0.868957,0.555274,0.001589,0.627098,0.825503,0.927217,0.570639,0.468580,0.114448,...,0.733671,0.928552,0.622528,0.958494,0.969182,0.856950,0.946895,0.151019,0.082922,0.877288
2,70,0.890677,0.647448,0.012695,0.742597,0.893300,0.939789,0.584597,0.453424,0.328950,...,0.847145,0.920582,0.616654,0.959841,0.961424,0.831121,0.151350,0.112606,0.806248,0.902736
3,132,0.823271,0.400048,0.013814,0.666755,0.793269,0.848218,0.541023,0.441119,0.382597,...,0.682520,0.938003,0.678210,0.960456,0.974422,0.878183,0.779329,0.232329,0.660918,0.871717
4,200,0.936625,0.528313,0.013414,0.845154,0.818526,0.937871,0.607533,0.574345,0.288392,...,0.769525,0.943947,0.662020,0.973221,0.973282,0.863142,0.434251,0.210331,0.615987,0.904432
